# Methodology

This notebook shows the methodology for the paper "Measuring OpenStreetMap building footprint completeness using human settlement layers".

## Setup

We import all of the relevant packages as well as download the datasets.

For reference, here are the original download links for the datasets:
1. High Resolution Settlement Layer (HRSL) ([Philippines](https://data.humdata.org/dataset/philippines-high-resolution-population-density-maps-demographic-estimates)) ([Madagascar](https://data.humdata.org/dataset/highresolutionpopulationdensitymaps-mdg))
2. Administrative Boundaries ([Philippines](https://data.humdata.org/dataset/philippines-administrative-levels-0-to-3)) ([Madagascar](https://data.humdata.org/dataset/madagascar-administrative-level-0-4-boundaries))
3. OpenStreetMap (OSM) ([Philippines](https://download.geofabrik.de/asia/philippines.html)) ([Madagascar](https://download.geofabrik.de/africa/madagascar.html))

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
import rasterio
import rasterio.features

import wget

import os
import glob
from zipfile import ZipFile

In [ ]:
try:
    os.mkdir("../download_data")
except Exception:
    pass

### HRSL download

Uncomment the cells below if you have not yet downloaded the HRSL datasets.

In [ ]:
# hrsl_mdg_url = "https://data.humdata.org/dataset/9e7ff424-7b9c-42cc-b869-5756fcad0956/resource/1fafdd04-8e0b-4c2a-b4dc-8f3ff39e3015/download/population_mdg_2018-10-01.zip"
# hrsl_phl_men_url = "https://data.humdata.org/dataset/6d9f35c0-4764-49ee-b364-329db0b7a47d/resource/5a13bb60-4506-42a5-a08a-7ccf20413179/download/phl_men_2019-06-01_geotiff.zip"
# hrsl_phl_women_url = "https://data.humdata.org/dataset/6d9f35c0-4764-49ee-b364-329db0b7a47d/resource/4aff438c-43d9-47d0-853f-5a6b6ae28223/download/phl_women_2019-06-01_geotiff.zip"

In [ ]:
# wget.download(hrsl_mdg_url, '../download_data/mdg_hrsl_oct_2018.zip')

In [ ]:
# wget.download(hrsl_phl_men_url, '../download_data/phl_hrsl_men_jun_2019.zip')

In [ ]:
# wget.download(hrsl_phl_women_url, '../download_data/phl_hrsl_women_jun_2019.zip')

### Admin boundary download

Uncomment the cells below if you have not yet downloaded the admin boundary datasets.

In [ ]:
# adm_mdg_url = "https://data.humdata.org/dataset/caf116df-f984-4deb-85ca-41b349d3f313/resource/12457689-6a86-4474-8032-5ca9464d38a8/download/phl_adm_psa_namria_20200529_shp.zip"
# adm_phl_url = "https://data.humdata.org/dataset/caf116df-f984-4deb-85ca-41b349d3f313/resource/12457689-6a86-4474-8032-5ca9464d38a8/download/phl_adm_psa_namria_20200529_shp.zip"

In [ ]:
# wget.download(adm_mdg_url, '../download_data/mdg_adm_all.zip')

In [ ]:
# wget.download(adm_phl_url, '../download_data/phl_adm_all.zip')

We note that the level 4 (barangay) admin boundary dataset for the Philippines is not available in the HDX website. We provide an external download link so that our results can still be reproduced.

In [ ]:
# adm4_phl_url = "https://storage.googleapis.com/osm-completeness-thinkingmachines/phl_adm_2015_level4_barangay.gpkg.zip"

In [ ]:
# wget.download(adm4_phl_url, '../download_data/phl_adm_2015_level4_barangay.gpkg.zip')

### OSM download

We note that the OSM download links used here are different from the OSM download links listed above. This is due to the methodology using OSM datasets from a previous date (Jan. 2020) and the OSM download links listed above only being available for the current date.

Uncomment the cells below if you have not yet downloaded the OSM datasets.

In [ ]:
# osm_mdg_url = "https://storage.googleapis.com/osm-completeness-thinkingmachines/mdg_osm_jan_2020_buildings.gpkg.zip"
# osm_phl_url = "https://storage.googleapis.com/osm-completeness-thinkingmachines/phl_osm_jan_2020_buildings.gpkg.zip"

In [ ]:
# wget.download(osm_mdg_url, '../download_data/mdg_osm_jan_2020_buildings.gpkg.zip')

In [ ]:
# wget.download(osm_phl_url, '../download_data/phl_osm_jan_2020_buildings.gpkg.zip')

### Unzip all datasets

In [ ]:
for i in glob.glob("../download_data/*.zip"):
    if os.path.isdir(os.path.splitext(i)[0]):
        pass
    else:
        with ZipFile(i) as myzip:
            myzip.extractall(os.path.splitext(i)[0])

## Get intersection of HRSL pixels and OSM buildings

We use Facebook’s High Resolution Settlement Layer (HRSL), a dataset of built-up areas derived from satellite images, as a proxy for ground truth building footprints. We then measure data completeness by getting the “percentage completeness” of pixels which is computed using the total percentage of pixels within the intersection of the human settlement layer and the OSM building footprints.

Pixels that intersect OSM buildings are *mapped*.

Pixels that do not intersect OSM buildings are *unmapped*.

### Madagascar (HRSL)

#### Load HRSL dataset

In [ ]:
# hrsl_mdg = rasterio.open(
#     "../download_data/mdg_hrsl_oct_2018/population_mdg_2018-10-01.tif"
# )

In [ ]:
# hrsl_mdg_crs = hrsl_mdg.crs

In [ ]:
# hrsl_mdg_band1_mask = hrsl_mdg.read_masks(1)

In [ ]:
# hrsl_mdg_rand = np.random.rand(
#     np.shape(hrsl_mdg_band1_mask)[0], np.shape(hrsl_mdg_band1_mask)[1]
# )
# hrsl_mdg_rand = hrsl_mdg_rand.astype("float32")

In [ ]:
# hrsl_mdg_band1_poly = list(
#     rasterio.features.shapes(
#         hrsl_mdg_rand, transform=hrsl_mdg.transform, mask=hrsl_mdg_band1_mask
#     )
# )

In [ ]:
# hrsl_mdg_geom = []
# for geom, value in hrsl_mdg_band1_poly:
#     geom = shapely.geometry.shape(geom)
#     hrsl_mdg_geom.append(geom)

In [ ]:
# hrsl_mdg_gdf = pd.DataFrame(hrsl_mdg_geom)
# hrsl_mdg_gdf = gpd.GeoDataFrame(hrsl_mdg_gdf, geometry=hrsl_mdg_gdf[0], crs="EPSG:4326")
# hrsl_mdg_gdf.drop(columns=[0], inplace=True)
# hrsl_mdg_gdf.reset_index(level=0, inplace=True)

In [ ]:
# hrsl_mdg_gdf.to_file('../data/hrsl_mdg.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [ ]:
hrsl_mdg_gdf = gpd.read_file('../data/hrsl_mdg.gpkg', driver='GPKG')

#### Load OSM dataset

In [ ]:
# osm_mdg = gpd.read_file(
#     "../download_data/mdg_osm_jan_2020_buildings.gpkg/mdg_osm_jan_2020_buildings.gpkg",
#     driver="GPKG",
# )

#### Get mapped pixels

In [ ]:
# mdg_pixels_with_buildings = gpd.sjoin(
#     hrsl_mdg_gdf, osm_mdg, how="inner", op="intersects"
# )

In [ ]:
# mdg_pixels_with_buildings = mdg_pixels_with_buildings.drop_duplicates(subset='index')

In [ ]:
# mdg_pixels_with_buildings.drop(columns=['index_right', 'osm_id', 'code', 'fclass', 'name', 'type'], inplace=True)

In [ ]:
# mdg_pixels_with_buildings.to_file('../data/test_mdg_pixels_with_buildings.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [ ]:
mdg_pixels_with_buildings = gpd.read_file('../data/mdg_pixels_with_buildings.gpkg', driver='GPKG')

#### Get unmapped pixels

In [ ]:
# mdg_pixels_no_buildings = pd.merge(hrsl_mdg_gdf, mdg_pixels_with_buildings, how='outer', indicator=True)

In [ ]:
# mdg_pixels_no_buildings = mdg_pixels_no_buildings[mdg_pixels_no_buildings['_merge'] == 'left_only']

In [ ]:
# mdg_pixels_no_buildings.drop(columns=['_merge'], inplace=True)

In [ ]:
# mdg_pixels_no_buildings.to_file('../data/mdg_pixels_no_buildings.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [ ]:
mdg_pixels_no_buildings = gpd.read_file('../data/mdg_pixels_no_buildings.gpkg', driver='GPKG')

#### Calculate percentage completeness

It's 10.89%.

In [ ]:
len(mdg_pixels_with_buildings) / (len(mdg_pixels_with_buildings) + len(mdg_pixels_no_buildings)) * 100

### Philippines (HRSL)

#### Load HRSL dataset

In [ ]:
# hrsl_phl_men = rasterio.open(
#     "../download_data/phl_hrsl_men_jun_2019/PHL_men_2019-06-01.tif"
# )

# hrsl_phl_women = rasterio.open(
#     "../download_data/phl_hrsl_women_jun_2019/PHL_women_2019-06-01.tif"
# )

In [ ]:
# hrsl_phl_crs = hrsl_phl_men.crs

In [ ]:
# hrsl_phl_men_band1_mask = hrsl_phl_men.read_masks(1)

In [ ]:
# hrsl_phl_women_band1_mask = hrsl_phl_women.read_masks(1)

In [ ]:
# hrsl_phl_band1_mask = hrsl_phl_men_band1_mask + hrsl_phl_women_band1_mask

In [ ]:
# hrsl_phl_rand = np.random.rand(
#     np.shape(hrsl_phl_band1_mask)[0], np.shape(hrsl_phl_band1_mask)[1]
# )
# hrsl_phl_rand = hrsl_phl_rand.astype("float32")

In [ ]:
# hrsl_phl_band1_poly = list(
#     rasterio.features.shapes(
#         hrsl_phl_rand, transform=hrsl_phl_men.transform, mask=hrsl_phl_band1_mask
#     )
# )

In [ ]:
# hrsl_phl_geom = []
# for geom, value in hrsl_phl_band1_poly:
#     geom = shapely.geometry.shape(geom)
#     hrsl_phl_geom.append(geom)

In [ ]:
# hrsl_phl_gdf = pd.DataFrame(hrsl_phl_geom)
# hrsl_phl_gdf = gpd.GeoDataFrame(hrsl_phl_gdf, geometry=hrsl_phl_gdf[0], crs="EPSG:4326")
# hrsl_phl_gdf.drop(columns=[0], inplace=True)
# hrsl_phl_gdf.reset_index(level=0, inplace=True)

In [ ]:
# hrsl_phl_gdf.to_file('../data/hrsl_phl.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [ ]:
hrsl_phl_gdf = gpd.read_file('../data/hrsl_phl.gpkg', driver='GPKG')

#### Load OSM dataset

In [ ]:
# osm_phl = gpd.read_file(
#     "../download_data/phl_osm_jan_2020_buildings.gpkg/phl_osm_jan_2020_buildings.gpkg",
#     driver="GPKG",
# )

#### Get mapped pixels

In [ ]:
# phl_pixels_with_buildings = gpd.sjoin(
#     hrsl_phl_gdf, osm_phl, how="inner", op="intersects"
# )

In [ ]:
# phl_pixels_with_buildings = phl_pixels_with_buildings.drop_duplicates(subset='index')

In [ ]:
# phl_pixels_with_buildings.drop(columns=['index_right', 'osm_id', 'code', 'fclass', 'name', 'type'], inplace=True)

In [ ]:
# phl_pixels_with_buildings.to_file('../data/phl_pixels_with_buildings.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [2]:
phl_pixels_with_buildings = gpd.read_file('../data/phl_pixels_with_buildings.gpkg', driver='GPKG')

#### Get unmapped pixels

In [ ]:
# phl_pixels_no_buildings = pd.merge(hrsl_phl_gdf, phl_pixels_with_buildings, how='outer', indicator=True)

In [ ]:
# phl_pixels_no_buildings = phl_pixels_no_buildings[phl_pixels_no_buildings['_merge'] == 'left_only']

In [ ]:
# phl_pixels_no_buildings.drop(columns=['_merge'], inplace=True)

In [ ]:
# phl_pixels_no_buildings.to_file('../data/phl_pixels_no_buildings.gpkg', driver='GPKG')

Just run this if you already ran the commands above.

In [3]:
phl_pixels_no_buildings = gpd.read_file('../data/phl_pixels_no_buildings.gpkg', driver='GPKG')

#### Calculate percentage completeness

It's 31.39%.

In [ ]:
len(phl_pixels_with_buildings) / (len(phl_pixels_with_buildings) + len(phl_pixels_no_buildings)) * 100

## Aggregate to different admin boundaries

### Philippines

#### Turn mapped pixels from a polygon layer to a point layer

In [4]:
phl_pixels_with_buildings['geometry'] = phl_pixels_with_buildings['geometry'].centroid

#### Turn unmapped pixels from a polygon layer to a point layer

In [5]:
phl_pixels_no_buildings['geometry'] = phl_pixels_no_buildings['geometry'].centroid

#### Level 1

##### *Load admin boundary*

In [ ]:
phl_adm1 = gpd.read_file('../download_data/phl_adm_all/phl_admbnda_adm1_psa_namria_20200529.shp')

##### *Count number of mapped pixels in admin boundary (level 1)*

In [ ]:
phl_adm1_sjoin_pixels_with_buildings = gpd.sjoin(
    phl_adm1, phl_pixels_with_buildings, how='left', op='intersects'
)

In [ ]:
phl_adm1_sjoin_pixels_with_buildings.dropna(subset=['index_right'], inplace=True)

In [ ]:
phl_adm1_count_pixels_with_buildings = pd.pivot_table(phl_adm1_sjoin_pixels_with_buildings, values=['index_right'], index=['ADM1_PCODE'], aggfunc=len)

In [ ]:
phl_adm1_count_pixels_with_buildings.rename(columns={'index_right': 'pixels_with_buildings'}, inplace=True)

In [ ]:
phl_adm1_count_pixels_with_buildings.to_csv('../data/phl_adm1_count_pixels_with_buildings.csv')

##### *Count number of unmapped pixels in admin boundary (level 1)*

In [ ]:
phl_adm1_sjoin_pixels_no_buildings = gpd.sjoin(
    phl_adm1, phl_pixels_no_buildings, how='left', op='intersects'
)

In [ ]:
phl_adm1_sjoin_pixels_no_buildings.dropna(subset=['index_right'], inplace=True)

In [ ]:
phl_adm1_count_pixels_no_buildings = pd.pivot_table(phl_adm1_sjoin_pixels_no_buildings, values=['index_right'], index=['ADM1_PCODE'], aggfunc=len)

In [ ]:
phl_adm1_count_pixels_no_buildings.rename(columns={'index_right': 'pixels_no_buildings'}, inplace=True)

In [ ]:
phl_adm1_count_pixels_no_buildings.to_csv('../data/phl_adm1_count_pixels_no_buildings.csv')

#### Level 4

##### *Load admin boundary*

In [6]:
phl_adm4 = gpd.read_file('../download_data/phl_adm_2015_level4_barangay.gpkg/phl_adm_2015_level4_barangay.gpkg')

##### *Create index for level 4 barangay admin boundary*

In [7]:
phl_adm4['ADM4_PCODE_NAME'] = phl_adm4['Bgy_Code'] + '_' + phl_adm4['Bgy_Name']

##### *Count number of mapped pixels in admin boundary (level 4)*

In [8]:
phl_adm4_sjoin_pixels_with_buildings = gpd.sjoin(
    phl_adm4, phl_pixels_with_buildings, how='left', op='intersects'
)

In [9]:
phl_adm4_sjoin_pixels_with_buildings.dropna(subset=['index_right'], inplace=True)

In [10]:
phl_adm4_count_pixels_with_buildings = pd.pivot_table(phl_adm4_sjoin_pixels_with_buildings, values=['index_right'], index=['ADM4_PCODE_NAME'], aggfunc=len)

In [11]:
phl_adm4_count_pixels_with_buildings.rename(columns={'index_right': 'pixels_with_buildings'}, inplace=True)

In [12]:
phl_adm4_count_pixels_with_buildings.to_csv('../data/phl_adm4_count_pixels_with_buildings.csv')

#### *Count number of unmapped pixels in admin boundary (level 4)*

In [13]:
phl_adm4_sjoin_pixels_no_buildings = gpd.sjoin(
    phl_adm4, phl_pixels_no_buildings, how='left', op='intersects'
)

In [14]:
phl_adm4_sjoin_pixels_no_buildings.dropna(subset=['index_right'], inplace=True)

In [15]:
phl_adm4_count_pixels_no_buildings = pd.pivot_table(phl_adm4_sjoin_pixels_no_buildings, values=['index_right'], index=['ADM4_PCODE_NAME'], aggfunc=len)

In [16]:
phl_adm4_count_pixels_no_buildings.rename(columns={'index_right': 'pixels_no_buildings'}, inplace=True)

In [17]:
phl_adm4_count_pixels_no_buildings.to_csv('../data/phl_adm4_count_pixels_no_buildings.csv')